In [8]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure
from bokeh.io import show, output_notebook, export_png
from sklearn.linear_model import LogisticRegression
output_notebook()

Loading BokehJS ...

In [6]:
data = pd.read_csv('notebooks/data/FoodData/ifood_df.csv')

In [7]:
data.head()

,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,marital_Together,marital_Widow,education_2n Cycle,education_Basic,education_Graduation,education_Master,education_PhD,MntTotal,MntRegularProds,AcceptedCmpOverall
0,58138.0,0,0,58,635,88,546,172,88,88,...,0,0,0,0,1,0,0,1529,1441,0
1,46344.0,1,1,38,11,1,6,2,1,6,...,0,0,0,0,1,0,0,21,15,0
2,71613.0,0,0,26,426,49,127,111,21,42,...,1,0,0,0,1,0,0,734,692,0
3,26646.0,1,0,26,11,4,20,10,3,5,...,1,0,0,0,1,0,0,48,43,0
4,58293.0,1,0,94,173,43,118,46,27,15,...,0,0,0,0,0,0,1,407,392,0


In [53]:
data['Response'] = data['AcceptedCmpOverall'].map({0:0,2:1,3:1,4:1})


In [54]:

data=data.dropna()
x=data['MntTotal'].values
y=data['Response'].values

In [56]:
f=figure()
f.scatter(x=x,y=y)
show(f)

In [59]:
L=LogisticRegression()
L.fit(x.reshape(-1,1),y)

LogisticRegression()

In [62]:
P=L.predict_proba(x.reshape(-1,1))[:,1]

In [64]:
g=figure()
g.scatter(x=x,y=P)
show(g)

In [65]:
L.score(x.reshape(-1,1),y)

0.935740839086564

In [68]:
describe(x)

NameError: name 'describe' is not defined

In [79]:
data['DollarRange']=pd.cut(data['MntTotal'],bins=np.arange(0,2500,100),labels=np.arange(0,2400,100))

In [80]:
data['DollarRange']

0       1500
1          0
2        700
3          0
4        400
        ... 
2198       0
2199       0
2200    1000
2203     700
2204     100
Name: DollarRange, Length: 1883, dtype: category
Categories (24, int64): [0 < 100 < 200 < 300 ... 2000 < 2100 < 2200 < 2300]

In [91]:
z=data.groupby('DollarRange')['Response'].mean()
n=data.groupby('DollarRange')['Response'].count()
u=L.predict_proba(100*np.arange(0,25).reshape(-1,1))
u.shape

(25, 2)

In [96]:
h=figure()
h.scatter(x=np.arange(0,25),y=z)
h.line(x=np.arange(0,25),y=u[:,1])
show(h)